# Chapter 4. 사전과 셋

**예제 4-1** 리스트를 이용한 전화번호 검색

In [10]:
def find_phonenumber(phonebook, name):
    for n, p in phonebook:
        if n == name:
            return p
    return None

phonebook = [
    ("John Doe", "555-555-5555"),
    ("Albert Einstein", "212-555-5555"),
]
print(f"John Doe's phone number is {find_phonenumber(phonebook, 'John Doe')}")
%timeit find_phonenumber(phonebook, 'John Doe')

John Doe's phone number is 555-555-5555
174 ns ± 7.52 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


**예제 4-2** 사전을 이용한 전화번호 검색

In [13]:
phonebook = {
    "John Doe": "555-555-5555",
    "Albert Einstein": "212-555-5555",
}
print(f"John Doe's phone number is {phonebook['John Doe']}")
%timeit phonebook['John Doe']

John Doe's phone number is 555-555-5555
47.8 ns ± 1.46 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


**예제 4-3** 리스트와 셋에서 유일한 이름 찾기

In [14]:
import random
import string
import timeit


def list_unique_names(phonebook):
    unique_names = []
    for name, phonenumber in phonebook:
        first_name, last_name = name.split(" ", 1)
        for unique in unique_names:
            if unique == first_name:
                break
        else:
            unique_names.append(first_name)
    return len(unique_names)


def set_unique_names(phonebook):
    unique_names = set()
    for name, phonenumber in phonebook:
        first_name, last_name = name.split(" ", 1)
        unique_names.add(first_name)
    return len(unique_names)


def random_name():
    first_name = "".join(random.sample(string.ascii_letters, 8))
    last_name = "".join(random.sample(string.ascii_letters, 8))
    return "{} {}".format(first_name, last_name)


if __name__ == "__main__":
    phonebook = [("John Doe", "555-555-5555"), ("Albert Einstein", "212-555-5555")]

    print("Number of unique names from set method:", set_unique_names(phonebook))
    print("Number of unique names from list method:", list_unique_names(phonebook))

    setup = (
        "from __main__ import (large_phonebook, set_unique_names, list_unique_names)"
    )
    iterations = 50
    large_phonebook = [(random_name(), "555-555-5555") for i in range(1000)]

    t = timeit.timeit(
        stmt="list_unique_names(large_phonebook)", setup=setup, number=iterations
    )
    print(
        f"Finding unique names in a phonebook of length {len(large_phonebook)} "
        f"using lists took: {t / iterations:2e} seconds"
    )

    t = timeit.timeit(
        stmt="set_unique_names(large_phonebook)", setup=setup, number=iterations
    )
    print(
        f"Finding unique names in a phonebook of length {len(large_phonebook)} "
        f"using sets took: {t / iterations:2e} seconds"
    )

Number of unique names from set method: 2
Number of unique names from list method: 2
Finding unique names in a phonebook of length 1000 using lists took: 1.827554e-02 seconds
Finding unique names in a phonebook of length 1000 using sets took: 2.616760e-04 seconds


## 4.1 사전과 셋의 동작원리

사전과 셋은 모두 해시 테이블을 사용해서 시간 복잡도가 O(1)이다.

### 4.1.1 삽입과 검색

해시 테이블을 처음 생성하면 배열을 사용할 때 처럼 메모리부터 할당한다. <br>
<br>
새로운 데이터의 위치는 데이터의 두 가지 속성으로 결정된다.
하나는 키의 해시값이고, 다른 하나는 데이터의 값을 다른 객체와 비교하는 방법이다.
데이터를 삽입하면 먼저 키를 해시한 후 마스크 처리하여 배열의 색인으로 만들기 때문이다. 
마스크는 해시값이 할당된 메모리 블록의 수보다 작아지도록 조정한다. 
만약에 메모리 블록을 8개 할당했고 해시값이 28975라면, 28975 & 0b111 = 7을 블록 색인으로 사용할 수 있을 것이다.
하지만 사전이 512개의 메모리 블록을 차지할 정도로 커진다면 마스크값은 ob111111111이 되고,
블록 색인은 28975 & ob111111111이 된다.

**예제 4-4** 사전 탐색 과정

In [ ]:
def index_sequence(key, mask=0b111, PERTURB_SHIFT=5):
    perturb = hash(key) # 1
    i = perturb & mask
    yield i
    while True:
        perturb >>= PERTURB_SHIFT
        i = (i * 5 + perturb + 1) & mask
        yield i

# 1: hash는 정수를 반환하며 C파이썬의 실제 C 코드는 부호가 없는 정수를 사용한다. 
#    따라서 이 의사 코드가 C파이썬의 동작을 100% 재현하지는 못하지만 어느 정도 가늠할 수는 있다.

**예제 4-5** 사용자 정의 해시 함수

In [ ]:
class City(str):
    def __hash__(self):
        return ord(self[0])

# 임의 값의 도시 이름으로 사전을 생성한다.
data = {
    City("Rome"): "Italy",
    City("San Francisco"): "USA",
    City("New York"): "USA",
    City("Barcelona"): "Spain",
}

# 해쉬 함수는 파이썬의 ord 함수를 사용한다. (입력의 첫 글자를 정수로 변환한다.)
# 항목이 4개 있는 사전에서 마스크값으로 0b111을 사용하는데, 다음과 같이 Barcelona와 Rome이 같은 색인을 만들어 낸다.
# (사전의 최소 항목 개수가 8개라서 마스크가 0b111이다.)
# City("Barcelona").__hash__() = ord("B") = 66이고, 66 & 0b111 = 2이다.
# City("Rome").__hash__() = ord("R") = 82이고, 82 & 0b111 = 2이다.

### 4.1.3 크기 변경

해시 테이블에 항목이 추가되면 해시 테이블의 크기도 그에 맞춰 변경되어야 한다. <br>
<br>
기본적으로 사전 혹은 셋의 최소 크기는 8이다. 그리고 사전이 2/3만큼 찰 때마다 크기를 3배 늘린다. 즉, 다음과 같은 순서로 크기가 변경된다. <br>
8; 18; 39; 81; 165; 333; 669; 1,341; 2,685; ... <br>
<br>
해시 테이블에서 많은 항목이 삭제되면 크기가 줄어들 수 있다. 하지만 **크기 변경은 삽인 연산 중에만 발생**한다.

### 4.1.4 해시 함수와 엔트로피

일반적으로 어떤 클래스의 두 인스턴스는 서로 다르고 해시 테이블 내에서 충돌이 발생하지 않으므로 이 내용을 받아들일 수 있다. 하지만 때에 따라서는 set이나 dict 객체가 항목의 id차이를 인식하지 않았으면 하기도 한다. 다음 클래스 정의를 보라.

In [19]:
class Point(object):
    def __init__(self, x, y):
        self.x, self.y = x, y

만일 x, y값이 동일한 Point객체를 여러 개 생성하면 메모리에서 각 객체는 서로 다른 위치에 있으므로 해시값이 모두 다르다. 
이 객체들을 모두 같은 set에 추가한다면 각각의 항목이 추가된다.

In [23]:
p1 = Point(1,1)
p2 = Point(1,1)
print(set([p1, p2]))
print(Point(1,1) in set([p1, p2]))

{<__main__.Point object at 0x000002D3C257CA00>, <__main__.Point object at 0x000002D3C257CC10>}
False


이 문제는 객체의 메모리 주소가 아니라 실제 내용을 기반으로 하는 사용자 정의 해시 함수를 작성해서 해결할 수 있다.

In [1]:
class PointHash(object):
    def __init__(self, x, y):
        self.x, self.y = x, y

    def __hash__(self):
        return hash((self.x, self.y))

    def __eq__(self, other):
        return self.x == other.x and self.y == other.y

In [4]:
p3 = PointHash(1,1)
p4 = PointHash(1,1)
print(set([p3, p4]))
print(PointHash(1,1) in set([p3, p4]))

{<__main__.PointHash object at 0x000002AFBF4F5970>}
True


해시 함수가 얼마나 균일한 분포로 해시값을 만들어 내는지를 해시 함수의 **엔트로피**라고 하며, 다음과 같이 정의한다. <br>
$$ S = -\sum_i p(i) \log p(i) $$
$p(i)$는 해시함수 i를 출력할 확률이다. 모든 해시값의 확률이 같을 때 엔트로피가 최대가 된다. 엔트로피가 최대가 되는 해시 함수는 최소 충돌을 보장하며 **완전** 해시 함수라고 한다.

**예제 4-6** 두 소문자를 조합한 최적 해시 함수

예를 들어, 사전의 키로 두 소문자의 모든 조합('aa', 'ab', 'ac' 등 총 676개)을 사용한다면 다음과 같은 해시 함수가 괜찮은 선택이 될 것이다. 이 해시 함수는 모든 두 소문자 조합에 대해서 해시 충돌이 발생하지 않으며, 항목 676개 짜리 사전에는 크기가 2,048인 해시 테이블이 사용되므로 마스크값은 bin(2048-1) = 0b1111111111을 사용하면 된다.

In [ ]:
def twoletter_hash(key):
    offset = ord('a')
    k1, k2 = key
    return (ord(k2) - offset) + 26 * (ord(k1) - offset)

**예제 4-7** 좋은 해시 함수와 나쁜 해시 함수의 시간 차이

In [5]:
import string
import timeit

class BadHash(str):
    def __hash__(self):
        return 42

class GoodHash(str):
    def __hash__(self):
        """
        This is a slightly optimized version of twoletter_hash
        """
        return ord(self[1]) + 26 * ord(self[0]) - 2619

if __name__ == "__main__":
    baddict = set()
    gooddict = set()
    for i in string.ascii_lowercase:
        for j in string.ascii_lowercase:
            key = i + j
            baddict.add(BadHash(key))
            gooddict.add(GoodHash(key))

    badtime = timeit.repeat(
        "key in baddict",
        setup="from __main__ import baddict, BadHash; key = BadHash('zz')",
        repeat=3,
        number=100_000,
    )
    goodtime = timeit.repeat(
        "key in gooddict",
        setup="from __main__ import gooddict, GoodHash; key = GoodHash('zz')",
        repeat=3,
        number=100_000,
    )

    print(f"Min lookup time for baddict: {min(badtime)}")
    print(f"Min lookup time for gooddict: {min(goodtime)}")

Min lookup time for baddict: 1.397932800000035
Min lookup time for gooddict: 0.030049799999915194


## 4.2 사전과 네임스페이스

파이썬에서 변수, 함수, 모듈이 사용될 때 그 객체를 어디서 찾을지 결정하는 계층이 있다. <br>
<br>
가장 먼저 모든 지역 변수를 담은 locals() 배열을 찾는다. 파이썬은 지역 변수 탐색을 빨리 끝내도록 최선을 다하며, 이과정은 사전 탐색을 하지 않는 유일한 부분이다. <br>
<br>
다음으로 globals() 사전에서 찾는다. <br>
<br>
마지막으로 __ builtin __ 객체에서 찾는다. locals()와 globals()는 사전이지만 __ builtin __ 은 기술적으로는 모듈 객체이다.

**예제 4-8** 네임스페이스 탐색

In [6]:
import math
from math import sin


def test1(x):
    """
    >>> %timeit test1(123_456)
    162 µs ± 3.82 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
    """
    res = 1
    for _ in range(1000):
        res += math.sin(x)
    return res


def test2(x):
    """
    >>> %timeit test2(123_456)
    124 µs ± 6.77 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
    """
    res = 1
    for _ in range(1000):
        res += sin(x)
    return res


def test3(x, sin=math.sin):
    """
    >>> %timeit test3(123_456)
    105 µs ± 3.35 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
    """
    res = 1
    for _ in range(1000):
        res += sin(x)
    return res


**예제 4-9** 네임스페이스 탐색 과정의 역어셈블

In [9]:
import dis

In [10]:
dis.dis(test1)
# 먼저 math 모듈을 로드하고 그다음에 이 모듈에서 sin 함수를 찾는다. (사전 탐색 두 번 발생)

 10           0 LOAD_CONST               1 (1)
              2 STORE_FAST               1 (res)

 11           4 LOAD_GLOBAL              0 (range)
              6 LOAD_CONST               2 (1000)
              8 CALL_FUNCTION            1
             10 GET_ITER
        >>   12 FOR_ITER                18 (to 32)
             14 STORE_FAST               2 (_)

 12          16 LOAD_FAST                1 (res)
             18 LOAD_GLOBAL              1 (math)
             20 LOAD_METHOD              2 (sin)
             22 LOAD_FAST                0 (x)
             24 CALL_METHOD              1
             26 INPLACE_ADD
             28 STORE_FAST               1 (res)
             30 JUMP_ABSOLUTE           12

 13     >>   32 LOAD_FAST                1 (res)
             34 RETURN_VALUE


In [11]:
dis.dis(test2)
# math 모듈에서 명시적으로 sin 함수를 임포트했으므로, 이 sin 함수는 전역 네임스페이스에서 직접 접근할 수 있다.
# math 모듈을 찾은 다음 모듈의 속성을 탐색하는 과정을 피할 수 있지만, 여전히 전역 네임스페이스에서 sin 함수를 찾아야 한다.

 21           0 LOAD_CONST               1 (1)
              2 STORE_FAST               1 (res)

 22           4 LOAD_GLOBAL              0 (range)
              6 LOAD_CONST               2 (1000)
              8 CALL_FUNCTION            1
             10 GET_ITER
        >>   12 FOR_ITER                16 (to 30)
             14 STORE_FAST               2 (_)

 23          16 LOAD_FAST                1 (res)
             18 LOAD_GLOBAL              1 (sin)
             20 LOAD_FAST                0 (x)
             22 CALL_FUNCTION            1
             24 INPLACE_ADD
             26 STORE_FAST               1 (res)
             28 JUMP_ABSOLUTE           12

 24     >>   30 LOAD_FAST                1 (res)
             32 RETURN_VALUE


In [12]:
dis.dis(test3)
# sin 함수를 키워드 인자로 받도록 하고 기본값을 math 모듈의 sin 함수로 지정했다.
# 여전히 math 모듈 안에서 이 함수를 찾아야 하지만, test3 함수가 최초로 정의될 때만 찾는다.

 32           0 LOAD_CONST               1 (1)
              2 STORE_FAST               2 (res)

 33           4 LOAD_GLOBAL              0 (range)
              6 LOAD_CONST               2 (1000)
              8 CALL_FUNCTION            1
             10 GET_ITER
        >>   12 FOR_ITER                16 (to 30)
             14 STORE_FAST               3 (_)

 34          16 LOAD_FAST                2 (res)
             18 LOAD_FAST                1 (sin)
             20 LOAD_FAST                0 (x)
             22 CALL_FUNCTION            1
             24 INPLACE_ADD
             26 STORE_FAST               2 (res)
             28 JUMP_ABSOLUTE           12

 35     >>   30 LOAD_FAST                2 (res)
             32 RETURN_VALUE
